In [ ]:
from fish.ephys.ephys import load, chop_trials, estimate_onset
from fish.util.fileio import load_image
from glob import glob
import numpy as np

In [ ]:
dirs = {}
dirs['ephys'] = 'W:/davis/data/ephys/20170531/5dpf_th1_cochcrxgfap_rgeco_f1_opto_11/5dpf_th1_cochcrxgfap_rgeco_f1_opto_11.10chFlt'
dirs['im'] = 'F:/Yu/20170531/fish1/20170531_1_11_th1cochr_gfaprgeco_5dpf_stimMO_20170531_234826/'



fnames = glob(dirs['im'] + 'TM*.stack')
fnames.sort()

ep_dat = load(dirs['ephys'])
cam_times = estimate_onset(ep_dat[2], 3.7, 100)[:len(fnames)]

In [ ]:
trials = chop_trials(ep_dat[4])
# for each trial onset look for the nearest cam time before the onset.
# for each offset, look for the nearest cam time after offset
def match_cam_time(events, cam, timing):
    from numpy import array, where
    tmp = []
    output = []
    for a in events:
        lags = array([a-b for b in cam])
        before = len(lags[lags > 0]) - 1
        after = before + 1
        
        if (before >= 0) and (after < len(cam)):
            if timing == 'pre':                
                output.append(before)
            if timing == 'post':
                output.append(after)    
    return array(output)

trials_cam = {}

for key, val in trials.items():
    pre = match_cam_time(val[0], cam_times, timing='pre')
    post = match_cam_time(val[1], cam_times, timing='post')
    trials_cam[key] = [pre, post]

In [ ]:
trials_cam

In [ ]:
def trigger_data(triggers, window, fnames):
    from fish.util.fileio import load_image
    from numpy import array
    ims_trial = []
    for trig in triggers:
        tr = []
        for t_w in window:        
            tr.append(load_image(fnames[trig + t_w]))
        ims_trial.append(tr)
        
    return array(ims_trial)

In [ ]:
baseline_pre = trigger_data(trials_cam[2.0][0], np.arange(-5, -1), fnames)
baseline_post = trigger_data(trials_cam[2.0][-1], np.arange(1, 5), fnames)

In [ ]:
import pyqtgraph as pq
%gui qt

In [ ]:
mean_baseline_change = baseline_post.mean((0,1)) - baseline_pre.mean((0,1))

In [ ]:
pq.image(mean_baseline_change)